In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
import google.generativeai as palm
from google.api_core import client_options as client_options_lib
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = os.getenv('SERPER_API_KEY')

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [9]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [11]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [13]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [15]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [17]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [19]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [21]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [23]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [25]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [27]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [29]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, as they are best equipped to handle this type of work.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL",
"context": "The Data Analyst should use statistical modeling and machine learning to identify trends and predict market movements for the selected stock AAPL. They will need to provide insights and alerts about significant market opportunities or threats for AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to identify trends and predict market movements acc

In [31]:
from IPython.display import Markdown
Markdown(result)

**Risk Analysis Report for AAPL Trading Strategies**

**Short-Term Momentum Trading:**
- **Potential Risks:**
  - High volatility in the market leading to potential losses.
  - Missing key trends and entering/exiting positions at the wrong time.
- **Mitigation Strategies:**
  - Constant monitoring of AAPL's stock price trends and technical indicators.
  - Setting stop-loss orders and securing profits at predetermined levels.
  
**Pairs Trading:**
- **Potential Risks:**
  - Correlation breakdown between AAPL's stock and selected related security.
  - Market conditions affecting the profitability of pairs trading.
- **Mitigation Strategies:**
  - Conducting continuous correlation analysis and adjusting trading positions accordingly.
  - Implementing a hedging strategy to reduce overall portfolio risk.

**Volatility Trading:**
- **Potential Risks:**
  - Implied volatility levels impacting options trading strategies.
  - Market news and events leading to unexpected price fluctuations.
- **Mitigation Strategies:**
  - Utilizing risk management techniques like position sizing and hedging.
  - Monitoring market news closely to anticipate changes in AAPL's stock price.

**Event-Driven Trading:**
- **Potential Risks:**
  - Market reactions to AAPL's earnings report not aligning with expectations.
  - Short-term trading opportunities based on unreliable market sentiment.
- **Mitigation Strategies:**
  - Analyzing market expectations and historical earnings performance for preparation.
  - Implementing options strategies like earnings straddles to profit from post-earnings movements.

In conclusion, to effectively execute the approved trading strategies for AAPL, it is crucial to be aware of the potential risks associated with each strategy and implement appropriate mitigation strategies. Continuous monitoring, risk management, and strategic adjustments are key to optimizing trading efficiency and maximizing returns while ensuring alignment with the user's risk tolerance and preferences for Day Trading.